### 函数矢量化

函数矢量化: 只能处理标量参数的函数经过函数矢量化后就可以接受数组参数, 对数组中每个元素执行相同处理.

numpy提供了vectorize函数, 可以把普通函数矢量化, 返回矢量化函数, 这样就可以直接处理数组参数.

案例:

```python
"""
demo03_vectorize.py 函数矢量化
"""
import numpy as np
import math as m

def f(a, b):
	r = m.sqrt(a**2 + b**2)
	return r
# 处理标量
print(f(3, 4))
# 处理矢量参数
a = np.array([3, 4, 5])
b = np.array([4, 5, 6])
# 把f函数矢量化
f_vec = np.vectorize(f)
print(f_vec(a, b))
print(np.vectorize(f)(a, b))

# 使用frompyfunc实现函数矢量化
# 2: 函数接收2个参数    1: 函数有1个返回值
f_func = np.frompyfunc(f, 2, 1)
print(f_func(a, b))
```

案例: 定义一种买进卖出策略, 通过历史数据判断这种策略是否可以实施.

```python
"""
demo04_profit.py  自定义投资策略, 求收益
"""
import numpy as np
import matplotlib.pyplot as mp
import datetime as dt
import matplotlib.dates as md

# 当numpy解析文本时,将会把第一列中的每个字符串
# 都传给函数进行处理, 将处理完毕后的返回值
# 转成需要的M8[D]类型
def dmy2ymd(dmy):
	dmy = str(dmy, encoding='utf-8')
	# 把dmy转成日期对象
	d = dt.datetime.strptime(dmy, '%d-%m-%Y')
	t = d.date()
	s = t.strftime('%Y-%m-%d')
	return s

# 加载文件
dates, opening_prices, highest_prices, \
	lowest_prices, closing_prices = np.loadtxt(
	'../da_data/aapl.csv', delimiter=',', 
	usecols=(1,3,4,5,6), unpack=True, 
	dtype='M8[D], f8, f8, f8, f8', 
	converters={1:dmy2ymd})

# 定义一种投资策略
# 每天开盘价*0.99买入, 收盘价就卖出
def profit(opening_price, highest_price,
		lowest_price, closing_price):
    buying_price = opening_price*0.99
    if lowest_price <= \
        buying_price <= highest_price:
        # 返回收益率的百分比
        return (closing_price-buying_price) * \
		        100 / buying_price
    return np.nan

# 调用函数 求得收益率  
profits = np.vectorize(profit)(opening_prices, 
	highest_prices, lowest_prices, closing_prices)
print(profits)
# 获取nan的掩码数组
nan = np.isnan(profits)
dates, profits = dates[~nan], profits[~nan]

# 绘制收盘价
mp.figure('profits', facecolor='lightgray')
mp.title('profits', fontsize=18)
mp.xlabel('Date', fontsize=14)
mp.ylabel('Price', fontsize=14)
mp.tick_params(labelsize=10)
mp.grid(linestyle=':')
# 设置主刻度定位器为每周一
ax = mp.gca()
ax.xaxis.set_major_locator(
	md.WeekdayLocator(byweekday=md.MO))
ax.xaxis.set_major_formatter(
	md.DateFormatter('%Y/%m/%d'))

# 把M8[D]转为matplotlib识别的date类型
dates = dates.astype(md.datetime.datetime)
mp.plot(dates, profits, 
	color='dodgerblue', linewidth=2,
	linestyle='-', label='profits')

mp.hlines(profits.mean(), dates[0], 
	dates[-1], color='orangered', 
	linewidth=2)

mp.legend()
# 自动格式化x轴的日期输出
mp.gcf().autofmt_xdate()
mp.show()
```

### 矩阵

矩阵是matrix类型的对象, 该类继承自numpy.ndarray. 任何针对ndarray的操作,对矩阵对象同样有效. 但是作为子类, 矩阵又结合了其自身的特点, 做了必要的扩充. 比如: 矩阵乘法 / 矩阵求逆等运算.

**矩阵对象的创建**

```python
# 构建矩阵对象 
# ary: 任何可以被解释为矩阵的数据结构
# copy: 是否复制数据 (True:复制一份副本数据)
m = np.matrix(ary, copy=True)

np.mat(ary)

np.mat('矩阵拼块规则字符串')
```

**矩阵的乘法运算**

```python
m3 = np.mat('1 2 3; 4 5 6')
print(m3)
print(m3 * 10)
print(m3 * m3.T)
print(m3.T * m3)
```

**矩阵的逆矩阵**

若两个矩阵A / B满足: AB = BA = E (E为单位矩阵). 则称A与B互为逆矩阵. 

单位矩阵E: 主对角线为1, 其他元素都为0.

```python
mi = m.I  
mi = np.linalg.inv(m)
```

矩阵求逆时, 若把方阵推广到非方阵, 则称为矩阵的**广义逆矩阵**.

**矩阵的应用**

假设一帮孩子和家长旅游, 去程坐大巴, 小孩票价3元, 大人票价3.2元, 共花了118.4; 回程坐火车, 小孩票价3.5元, 大人票价3.6元, 共花了135.2. 分别求小孩和大人的人数.

```python
"""
demo06_p.py 应用题
"""
import numpy as np

prices = np.mat('3 3.2; 3.5 3.6')
totals = np.mat('118.4; 135.2')
x = np.linalg.lstsq(prices, totals)[0]
print(x)

x = prices.I * totals
print(x)
```

**解方程组**
$$
\begin{cases}
x-2y+z=0 \\
2y-8z-8=0 \\
-4x+5y+9z+9=0 \\
\end{cases}
$$

```
x = np.linalg.lstsq(A, B)
x = np.linalg.solve(A, B)
```

整理成矩阵相乘的形式:
$$
\left[ \begin{array}{c}
1 & -2 & 1 \\
0 & 2 & -8 \\
-4 & 5 & 9 \\
\end{array} \right]
\times
\left[ \begin{array}{c}
x \\
y \\
z \\
\end{array} \right]
=
\left[ \begin{array}{c}
0 \\
8 \\
-9 \\
\end{array} \right]
$$
案例:

```python
A = np.mat('1 -2 1; 0 2 -8; -4 5 9')
B = np.mat('0; 8; -9')
print(np.linalg.solve(A, B))
```

案例: 求斐波那契数列

1  1   2   3   5    8    13    21   .....

```
x	  1 1   1 1   1 1   
	  1 0   1 0   1 0  
----------------------------------
1 1   2 1   3 2   5 3
1 0   1 1   2 1   3 2  ...
```

案例:

```python
m = np.mat('1 1; 1 0')
for i in range(1, 30):
	print((m**i)[0,1], end=' ')
```

**傅里叶定理**

法国科学家傅里叶说过, 任何一个周期函数都是n个不同振幅/不同频率/不同相位的正弦函数叠加而成.

**合成方波**

一个方波由如下参数的正弦波叠加而成:
$$
y = 4\pi \times sin(x) \\
y = \frac{4}{3}\pi \times sin(3x) \\
y = \frac{4}{5}\pi \times sin(5x) \\
....\\
y = \frac{4}{2n-1}\pi \times sin((2n-1)x) \\
$$
案例:

```python
"""
demo08_sin.py  叠加方波
"""
import numpy as np
import matplotlib.pyplot as mp

x = np.linspace(-2*np.pi, 2*np.pi, 1000)
# 叠加3条曲线
y1 = 4*np.pi * np.sin(x)
y2 = 4/3*np.pi * np.sin(3*x)
y3 = 4/5*np.pi * np.sin(5*x)
y = y1 + y2 + y3
# 叠加1000条曲线
y = np.zeros(x.size)
for i in range(1, 1000):
	y += 4/(2*i-1)*np.pi * np.sin((2*i-1)*x)

mp.grid(linestyle=':')
mp.plot(x, y1)
mp.plot(x, y2)
mp.plot(x, y3)
mp.plot(x, y,linewidth=2)
mp.show()


```

### 特征值与特征向量

对于n阶方阵A, 如果存在数a和非零n维列向量x, 使得Ax=ax, 则称a是矩阵A的一个特征值, x是矩阵A属于特征值的特征向量.

```python
# 提取方阵A的特征值与特征向量
# eigvals: 一组特征值
# eigvecs: 特征值对应的特征向量
eigvals, eigvecs = np.linalg.eig(A)
# 通过特征值与特征向量 逆向推导原方阵
S = eigvecs * np.diag(eigvals) * eigvecs.I
```

案例:

```python
"""
demo01_eig.py 特征值与特征向量
"""
import numpy as np

A = np.mat('1 5 8; 2 5 7; 8 2 4')
# 提取特征值与特征向量
eigvals, eigvecs = np.linalg.eig(A)
print(eigvals)
print(eigvecs)
# 逆向推导原方阵
S = np.mat(eigvecs) * \
	np.mat(np.diag(eigvals)) * \
	np.mat(eigvecs).I
print(S)
eigvals[1:] = 0
S = np.mat(eigvecs) * \
	np.mat(np.diag(eigvals)) * \
	np.mat(eigvecs).I
print(S)
```

### 奇异值分解

有一个矩阵M, 可以分解为3个矩阵U S V, 使得UxSxV等于M. U与V都是正交矩阵(乘以自身的转置矩阵结果为单位矩阵). 那么S矩阵主对角线上的元素称为矩阵M的奇异值, 其他元素都为0.

```python
# 奇异值分解  U与V都是正交矩阵
# s: 奇异值数组
U, s, V = np.linalg.svd(M)
# 逆向推导原矩阵:
S = U * np.diag(s) * V
```

案例: 读取图片亮度矩阵,提取奇异值.保留部分奇异值,重新生成图片.

```python
# 奇异值分解 
U, s, V = np.linalg.svd(image)
S2 = np.mat(U)*np.mat(np.diag(s))*np.mat(V)
# 保留部分奇异值  生成图片
s[100:] = 0
S3 = np.mat(U)*np.mat(np.diag(s))*np.mat(V)

mp.subplot(2,2,3)
mp.xticks([])
mp.yticks([])
mp.imshow(S2, cmap='gray')
mp.subplot(2,2,4)
mp.xticks([])
mp.yticks([])
mp.imshow(S3, cmap='gray')
```

### 快速傅里叶变换(fft)

**什么是傅里叶变换?**

傅里叶定理: 任何一条周期曲线, 无论多么跳跃或不规则, 都能表示成一组光滑正弦曲线叠加之和. 傅里叶变换即是把这条周期曲线拆解成一组光滑正弦曲线的过程.

傅里叶变换的目的是将时域(时间域)上的信号转变为频域(频率域)上的信号, 随着域的不同,对同一个事物的了解角度也随之改变. 因此在时域中某些不好处理的地方, 放在频域中就可以较为简单的处理. 这样可以大量减少处理的数据量.

傅里叶定理:
$$
y = A_1sin(\omega_1x+\phi_1) + 
A_2sin(\omega_2x+\phi_2) + .. +C
$$
快速傅里叶变换相关API:

```python
import numpy.fft as nf
# 通过采样数与采样周期,得到fft分解所得曲线的频率数组
# 采样周期: x轴相邻两点的距离
freqs = nf.fftfreq(采样数量, 采样周期)

# 通过原函数值序列, 经过fft后, 得到复数数组
# 复数数组的长度即是拆解出正弦函数的个数
# 复数数组每个元素的模,代表每个正弦曲线的振幅
# 复数数组每个元素的辅角,代表每个正弦曲线的相位角
复数序列 = nf.fft(原函数值序列)

# 逆向傅里叶变换
原函数值序列 = nf.ifft(复数序列)
```

案例: 基于傅里叶变换, 拆解方波.

```python
"""
demo04_fft.py  傅里叶变换
"""
import numpy as np
import matplotlib.pyplot as mp
import numpy.fft as nf

x = np.linspace(-2*np.pi, 2*np.pi, 1000)

# 叠加1000条曲线
y = np.zeros(x.size)
for i in range(1, 1000):
	y += 4/(2*i-1)*np.pi * np.sin((2*i-1)*x)

# 对y做傅里叶变换, 绘制频域图像
ffts = nf.fft(y)
# 获取傅里叶变换的频率序列
freqs = nf.fftfreq(x.size, x[1]-x[0])
pows = np.abs(ffts)

mp.figure('FFT', facecolor='lightgray')
mp.subplot(121)
mp.grid(linestyle=':')
mp.plot(x, y,linewidth=2)
mp.subplot(122)
mp.grid(linestyle=':')
mp.plot(freqs[freqs>0], pows[freqs>0], 
	    c='orangered')
# 通过复数数组,经过ifft操作, 得到原函数
y2 = nf.ifft(ffts)
mp.subplot(121)
mp.plot(x, y2, linewidth=7, alpha=0.5)

mp.show()
```

**基于傅里叶变换的频域滤波**

含噪信号是高能信号与低能噪声叠加的信号, 可以通过傅里叶变换的频域滤波实现简单降噪.

通过FFT使含噪信号转换为含噪频谱, 手动取出低能噪声, 留下高能频谱后,再通过IFFT生成高能信号. 

1. 读取音频文件, 获取音频的基本信息: 采样个数/采样周期/每个采样点的声音值.  绘制音频的时域: 时间/位移图像.

```python
sample_rate, noised_sigs = \
	wf.read('../da_data/noised.wav')
print(noised_sigs.shape) 
times = np.arange(len(noised_sigs))/sample_rate

mp.figure('Filter', facecolor='lightgray')
mp.subplot(2,2,1)
mp.title('Time Domain', fontsize=16)
mp.ylabel('Signal', fontsize=12)
mp.tick_params(labelsize=8)
mp.grid(linestyle=':')
mp.plot(times[:178], noised_sigs[:178], 
	c='dodgerblue', label='Noised Sigs')
mp.legend()
mp.show()

```

2. 基于傅里叶变换, 获取音频频域信息, 绘制频域: 频率/能量图像.

```python
freqs = nf.fftfreq(times.size, 1/sample_rate)
noised_ffts = nf.fft(noised_sigs)
noised_pows = np.abs(noised_ffts)
mp.subplot(222)
mp.title('Frequency Domain', fontsize=16)
mp.ylabel('Power', fontsize=12)
mp.tick_params(labelsize=8)
mp.grid(linestyle=":")
mp.semilogy(freqs[freqs>0], noised_pows[freqs>0],
	c='orangered', label='Noised')
mp.legend()
```

3. 将低频噪声去除后绘制音频频域: 频率/能量图像.

```python
noised_inds = np.where(freqs != fund_freq)
filter_ffts = noised_ffts.copy()
filter_ffts[noised_inds] = 0
filter_pows = np.abs(filter_ffts)
mp.subplot(224)
mp.title('Frequency Domain', fontsize=16)
mp.ylabel('Power', fontsize=12)
mp.tick_params(labelsize=8)
mp.grid(linestyle=":")
mp.plot(freqs[freqs>0], filter_pows[freqs>0],
	c='orangered', label='Filter')
mp.legend()
```

4. 基于逆向傅里叶变换,生成时域的音频信号, 绘制时域: 时间/位移图像.

```python
filter_sigs = nf.ifft(filter_ffts)
mp.subplot(2,2,3)
mp.title('Time Domain', fontsize=16)
mp.ylabel('Signal', fontsize=12)
mp.tick_params(labelsize=8)
mp.grid(linestyle=':')
mp.plot(times[:178], filter_sigs[:178], 
	c='dodgerblue', label='Filter Sigs')
mp.legend()
```

5. 生成音频文件

```python
wf.write('../da_data/filter.wav', sample_rate, filter_sigs.astype(np.int16))
```

### 随机数模块

生成服从特定统计规律的随机数序列.

#### 二项分布(binomial)

二项分布是重复n次独立试验的伯努利试验.  每次试验只有两种可能的结果, 而且两种结果发生与否相互独立相互对立. 事件发生与否的概率在每一次独立试验中都保持不变.

```python
# 产生size个随机数, 每个随机数来自n次尝试中的成功
# 的次数, 其中每次尝试成功的概率为p
# 这样产生的随机数即符合二项分布规律.
np.random.binomial(n, p, size)
```

案例: 某人投篮命中率0.3, 投10次, 进5个的概率.

```python
# 某人投篮命中率0.3, 投10次, 进5个的概率.
r = np.random.binomial(10, 0.3, 100000)
p = (r==5).sum() / r.size
print(p)
```


#### 超几何分布

```python
# 产生size个随机数, 每个随机数为在总样本中随机抽取
# nsample个样本后,好样本的个数.
# 总样本由ngood个好样本和nbad个坏样本组成.
np.random.hypergeometric(
    ngood, nbad, nsample, size)
```

案例: 7个好苹果, 3个坏苹果, 抽出3个苹果, 求抽出2个好苹果的概率.

```python
# 7个好苹果, 3个坏苹果, 抽出3个苹果, 
# 求抽出2个好苹果的概率
r = np.random.hypergeometric(7, 3, 3, 100000)
p = (r==2).sum() / r.size
print(p)
```

#### 正态分布(normal)

```python
# 产生符合标准正态分布的size个随机数
# 期望=0, 标准差=1
np.random.normal(size)
# 产生符合正态分布的size个随机数
# 期望=1, 标准差=10
np.random.normal(loc=1, scale=10, size)
```
